In [ ]:
# setting notebook source
with open('./config_path.py') as f:
    exec(f.read())

In [ ]:
from config import secrets, settings
from zoho_api import *
import pandas as pd
import copy
from datetime import date

In [ ]:
backup_data = {}
response_data = {}

In [ ]:
refresh_token()

In [ ]:
SLAs = {}
SLAs['SupportME Premium'] = '10977000006774075'
SLAs['SupportME Comfort'] = '10977000006804083'
SLAs['SupportME Standard'] = '10977000006804131'
SLAs['SupportME StartUpCare'] = '10977000000007203'
SLAs['Default'] = '10977000000007201'
SLAs

In [ ]:
SupportHours = {}
SupportHours['SupportME Premium'] = 204
SupportHours['SupportME Comfort'] = 96
SupportHours['SupportME ComfortPlus'] = 8
SupportHours['SupportME Standard'] = 36
SupportHours[None] = None
SupportHours

In [ ]:
field_list = ['id'
              , 'accountName'
              , 'cf_account_id'
              , 'website'
              , 'cf_account_id'
              , 'cf_support_package'
              , 'cf_support_subsription_expiry_date'
              , 'associatedSLAIds'
              , 'cf_max_support_time'
              , 'cf_picklist_1'  # Customer type
              # , 'cf_loyality_program'
             ]

## App

In [ ]:
# Current data from Zoho
data_source = list_accounts(view='OEM customers', field_list=field_list)

In [ ]:
# Current data from Zoho
test = list_accounts(view='SI Elite customers', field_list=field_list)

In [ ]:
test

In [ ]:
# Data to be updated
data_update = pd.read_excel(r"../ListaOEMdoZOHO.xlsx" )

In [ ]:
update_df = data_update[data_update['Support Package'] == 'Standard']
len(update_df)

In [ ]:
df = pd.json_normalize(data_source)

In [ ]:
# joining data to have Zoho Account Reference Id and required new information in one table
joined_df = update_df.set_index('Zoho SAP ID').join(df.set_index('cf.cf_account_id')).reset_index()

In [ ]:
columns_to_keep = ['Zoho SAP ID',
 'Zoho Account Name',
 'Country (Account)',
 'Customer type',
 'Loyality Program',
 'Sales region',
 'Support Package',
 'Region',
 'id',
 'accountName']

In [ ]:
joined_df = joined_df[columns_to_keep]

In [ ]:
joined_df.head()

In [ ]:
len(joined_df[joined_df['id']==''])

In [ ]:
data_source[0]

In [ ]:
accounts_updated = copy.deepcopy(data_source)
modifications = {}
for account in accounts_updated:
    support_package = ''
    record = joined_df[joined_df['id']==account['id']]
    
    if len(record) == 0: #no need for update
        continue
    elif len(record) == 1: # there is record to update
        support_package = 'SupportME Standard'
        account['cf']['cf_support_subsription_expiry_date'] = '2026-04-30'
    else: # error in Zoho Data
        raise DuplicatedAccountsError

    # if any modification have been applied on data, prepare setup
    if support_package != '':   
        zoho_id = account.pop('id')
        account.pop('accountName')
        account.pop('associatedSLAIds')
        account.pop('website')
        account['cf'].pop('cf_account_id')
        account['cf'].pop('cf_picklist_1')
        account['cf']['cf_support_package'] = support_package
        account['cf']['cf_max_support_time'] = SupportHours[support_package]
        # account['website'] = support_package
        modifications[zoho_id] = account

In [ ]:
modifications

In [ ]:
processedIds = set()
missingIds = set()

In [ ]:
refresh_token()

for index, record in enumerate(modifications.items()):
    if record[0] in processedIds:
        continue
    account = backup_updated_fields(record[0], record[1], backup_data, response_data)
    if account is None:
        missingIds.add(record[0])
    else:
        try:
            account, response = update_account(record[0], record[1])
            if account is None or response.status_code != 200:
                missingIds.add(record[0])
                print(f"\r{index + 1}) Error for updating account: {record[0]}. \t\t")
            else:
                support_package = record[1]['cf']['cf_support_package']
                print(f"{index + 1}) Data for {record[0]} have been updated.\t", end='')
                apply_sla(record[0], SLAs[support_package])
                processedIds.add(record[0])
        except Exception as e:   
            missingIds.add(record[0])
            print(f"\r{index + 1}) Error for executing script for account: {record[0]}: {e}")
print("Updating process have been finished")

In [ ]:
len(processedIds)

In [ ]:
record[1]

In [ ]:
updated_fields = record[1]
today = date.today()
response_filename = "response_" +str(today) + "_SLA_OEM.json"
backup_filename = "backup_" +str(today) + "_SLA_OEM.json"
save_response_to_file(response_data = response_data,  filename=response_filename, updated_fields=updated_fields)
save_backup_to_file(backup_data, filename=backup_filename, updated_fields=updated_fields)